Ignore the over-annotated code, it's more brain-dumping.

# List of Tumbling Blocks

1. **Not adding type annotations to my dataclass**. This makes the class-level default values rather than instance attributes that can be set via the constructor.

# Setup

In [22]:
import gymnasium as gym
import numpy as np
import torch as t
import torch.nn as nn
import torch.optim as optim
import wandb
import warnings
import time
import einops

from dataclasses import dataclass
from gymnasium.spaces import Box, Discrete
from jaxtyping import Bool, Float, Int
from torch import nn, Tensor
from torch.distributions.categorical import Categorical
from torch.optim.optimizer import Optimizer
from tqdm import tqdm, trange
from typing import Literal

warnings.filterwarnings("ignore")
Arr: np.ndarray

device = t.device("mps") if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else t.device("cpu")
print(f"Using device: {device}")

Using device: mps


In [ ]:
# effectively @dataclass is always going to be paired with the Args class since it initalizes all of the arguments
# and gets rid of the the annoying initalizations in def __init__(self, )
@dataclass 
class PPOArgs:
    seed: int = 1
    env_id: str = "CartPole-PPO"
    mode: Literal["classic-control", "atari", "mujoco"] = "classic-control"

    total_timesteps: int = 500000
    num_envs: int = 4
    num_steps_per_rollout: int = 128
    num_minibatches: int = 4
    batches_per_learning_phase: int = 4

    lr: float = 2.5e-4
    max_grad_norm: float = 0.5

    gamma: float = 0.99
    gae_lambda: float = 0.95
    clip_coef: float = 0.2
    ent_coef: float = 0.01
    vf_coef: float = 0.25

    video_log_freq: int | None = None
    wandb_project_name: str = "Cartpole_PPO"
    wandb_entity: str = None

    # comments in reference to num_minibatches = 2
    def __post_init__(self):
        self.batch_size = self.num_steps_per_rollout * self.num_envs # 512

        self.minibatch_size = self.batch_size // self.num_minibatches # 256 
        self.total_phases = self.total_timesteps // self.batch_size # 976
        self.total_training_steps = self.total_phases * self.batches_per_learning_phase * self.num_minibatches # 7808

        self.video_save_path = "videos/cartpole_ppo"

args = PPOArgs(num_minibatches = 2)